In [37]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment3.ipynb")

# DS 453 / 653: Programming Assignment 3

**Due date**: Thursday, Feburary 15 at 8pm on [Gradescope](https://www.gradescope.com/courses/710247). Note that Programming Assignment 4 will be due at the same time, so plan your work accordingly.

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this assignment._

This assignment contains 3 questions worth a total of 5 points. You must receive at least 4 out of the available 5 points to pass the assignment.

## Assignment Overview

The goal of this assignment is to learn about digital signatures, hash functions, and the structure of a blockchain.

To do so, we introduce a toy blockchain that contains many of the same concepts as used within Bitcoin, but with some components removed (like the mining rewards, transaction fees, scripting language, and adjustments to the difficulty level).

### Python classes
Our toy blockchain is written as Python code, which we provide below. We separate the blockchain into four classes:
- The `TransactionPayload` class holds the relevant information for a single payment: the sender and receiver (identified by their public keys) and the amount of money being transferred.
- The `Transaction` class contains the payload together with a digital signature made by the sender.
- The `Block` class contains the block "height" (which is just a 0-indexed counter), a set of new transactions, and a pointer to the previous block in the chain. Additionally, the block contains a `nonce` that can be chosen arbitrarily by the block's miner in order to satisfy the difficulty rule.

Finally, the `blockchain` object is a list of blocks.

Throughout this assignment, the difficulty rule is that:
> _The hash of a valid block must begin with 8 bits (one byte) of all 0s_.

To instantiate these Python classes, execute the code blocks below.

In [38]:
import otter
grader = otter.Notebook()

In [39]:
## Execute, but DO NOT MODIFY this code block. ##
## It contains a Python class corresponding to a toy blockchain.
## Please read the code to understand how the toy blockchain works.

import json
from Crypto.Hash import SHA256
from Crypto.PublicKey import ECC
from Crypto.Signature import DSS
from binascii import hexlify, unhexlify

class Transaction():
  def __init__(self, transaction_payload, signature):
    self.transaction_payload = transaction_payload
    self.signature = signature

  def json_encode(self):
    transaction_json = {"transaction": json.loads(self.transaction_payload.json_encode()), "signature":self.signature}
    return json.dumps(transaction_json)

class TransactionPayload():
  def __init__(self, sender_public_key, receiver_public_key, amount):
    self.sender_public_key = sender_public_key
    self.receiver_public_key = receiver_public_key
    self.amount = amount

  def json_encode(self):
    return json.dumps(self.__dict__, sort_keys=True)

  def encode(self):
    # encodes the transaction as bytes
    return self.json_encode().encode('ascii')

  def hash(self):
    # returns the hash of the transaction as bytes
    return SHA256.new(self.encode())

class Block():
  def __init__(self, height, transactions, previous_hash, nonce=0):
    self.height = height
    self.transactions = transactions
    self.previous_hash = previous_hash
    self.nonce = nonce

  def json_encode(self):
    json_encoding = {'height': self.height,
            'transactions': [json.loads(transaction.json_encode()) for transaction in self.transactions],
            'previous_hash': self.previous_hash, # in hexstring
            'nonce': self.nonce
    }
    return json.dumps(json_encoding)

  def encode(self):
    return self.json_encode().encode('ascii')

  def hash(self):
    return SHA256.new(self.encode()).hexdigest()


### Participants in the toy blockchain

There are five participants who use this toy blockchain. We label these participants as players A, B, C, D, and E. We will only look at a few blocks of this blockchain, which operate as follows:

- Before the start of the blockchain, we assume that players A and C start with 1 coin. The other players have no money at the start.
- The first block contains two transactions: A pays 1 coin to B, and C pays 1 coin to D.
- The second block contains a single transaction: D pays 1 coin to A

- A -> B: 1 coin
- C -> D: 1 coin
- D -> A: 1 coin

After these two blocks, players A and B should have 1 coin each and everyone else should have nothing.

Execute the Python code below to instantiate this blockchain.

In [40]:
## Execute, but DO NOT MODIFY this code block. ##
## This code creates a concrete blockchain containing the transactions and blocks described above.
## It also contains all 5 players' public keys, and the corresponding secret key *only* for Player A.

# Player A's secret key
secret_key = ECC.generate(curve='P-256', randfunc=(lambda x: '0'.encode('ascii') * x))
# Public keys for all five players
# Player A's public key is located at index 0, player B's public key is at index 1, and so on
public_keys = ['044d4385fe08e0eb94c524bdd3682c9c5ae358f52402df02418d0ba6cf6889289a27bd56a666df7dc595c98da1f22958009ae17c52fd290185d2e2bf11140a0a5e', '04be4a2555cc8ed29989eeddde3d8e4d41458d02dce047aac2e6af2d58688c5beae7163eb157d1fd3c3d282abab7172b0f0d63274ebe721d31c5d72a83741df170', '04a3fc575afc4ad2ca2cbeed50b52fedf049d317b790aa33e7f8182aec028453082d4c4ffd4c99c7e6d11c134e81020c57ca0fbf0bf3406bf4457dc72d8c994a65', '04abc5bd2786ea5144122ccf43b89a557e1d36e2773c2b8986d9819a22c2e6540b3d5da692504ccc29fbf774435312afdda2a360d5160329cb326f6d47db29f50b', '046a921bed68e07e66f38a5137b3784372cfd4507e6451e1dfe3760f08649750e7891cd30339b730c0280738c60233d3c99eb78454340ff363cafbf1e6f0cfce5e']

blockchain = []

# Remember: we assume that A and C magically begin the blockchain with 1 coin.
# This is because our toy blockchain doesn't have mining rewards,
# so we make this assumption to start the chain with some money.

# Here are the transactions in each block
def gen_block_0():
  # This block sends 1 coin from public key A to public key B
  t1 = TransactionPayload(public_keys[0], public_keys[1], 1)
  # In this toy blockchain, all signatures are stored in hex format and have a type of string
  t1_signature = "edd6bb60e8486f9747f911a6d7340c275d5a417c3823952715a18a64d3577325413cf544e8f48e6c30a312649200c57b72d7eaa4721abf376dbd0dc799db6c3a"
  # A transaction includes the payload and its corresponding signature
  t1_signed = Transaction(t1, t1_signature)

  # C -> D
  t2 = TransactionPayload(public_keys[2], public_keys[3], 1)
  t2_signature = "33bcaaa920207125ec72ea7f9e5fbddb993479dbe7122c6dfcf0a96f2e646dd90ff73c90c784f473480e32fd2fb68735af86458bc0434e5958f025cc9e0ddd59"
  t2_signed = Transaction(t2, t2_signature)

  # block number 0
  previous_hash = "0000000000000000000000000000000000000000000000000000000000000000"
  b0 = Block(0, [t1_signed, t2_signed], previous_hash, 7)
  return b0

def gen_block_1():
  # D -> A
  t3 = TransactionPayload(public_keys[3], public_keys[0], 1)
  t3_signature = "886de552f43010b2a848f660b066102e680a4c6b763032c98c53e6f86c5d5638a9dee5e41f311e4956cd416a51850afa28bc4b9575cbf9cbf1940fae84b1d166"
  t3_signed = Transaction(t3, t3_signature)

  # block number 1
  previous_hash = "00fd71161f8b5244d0641d83ccaedc2f0b7bed9d3f0c625fc817218660c4369a"
  b1 = Block(1, [t3_signed], previous_hash, 143)
  return b1

def pretty_print(blockchain):
  [print(block.json_encode()) for block in blockchain]

blockchain = [gen_block_0(), gen_block_1()]
pretty_print(blockchain)


{"height": 0, "transactions": [{"transaction": {"amount": 1, "receiver_public_key": "04be4a2555cc8ed29989eeddde3d8e4d41458d02dce047aac2e6af2d58688c5beae7163eb157d1fd3c3d282abab7172b0f0d63274ebe721d31c5d72a83741df170", "sender_public_key": "044d4385fe08e0eb94c524bdd3682c9c5ae358f52402df02418d0ba6cf6889289a27bd56a666df7dc595c98da1f22958009ae17c52fd290185d2e2bf11140a0a5e"}, "signature": "edd6bb60e8486f9747f911a6d7340c275d5a417c3823952715a18a64d3577325413cf544e8f48e6c30a312649200c57b72d7eaa4721abf376dbd0dc799db6c3a"}, {"transaction": {"amount": 1, "receiver_public_key": "04abc5bd2786ea5144122ccf43b89a557e1d36e2773c2b8986d9819a22c2e6540b3d5da692504ccc29fbf774435312afdda2a360d5160329cb326f6d47db29f50b", "sender_public_key": "04a3fc575afc4ad2ca2cbeed50b52fedf049d317b790aa33e7f8182aec028453082d4c4ffd4c99c7e6d11c134e81020c57ca0fbf0bf3406bf4457dc72d8c994a65"}, "signature": "33bcaaa920207125ec72ea7f9e5fbddb993479dbe7122c6dfcf0a96f2e646dd90ff73c90c784f473480e32fd2fb68735af86458bc0434e5958f025cc9e0

This `blockchain` turns out to be valid because:
1. The signature in every transaction is correct,
2. Every block has a `nonce` that is chosen to satisfy the difficulty rule,
3. Every block points to the previous block (and by convention, the initial block -- called the "genesis" block -- has a hash of all-0s), and
4. Everyone has enough money in their account at the moment they post a transaction.

If we break any of the four rules, then the blockchain becomes invalid. Here are some examples of invalid blockchains.

### Your Objective

Your objective in this assignment is to determine whether any given blockchain is valid according to the 4 rules stated above. If so, you must find the balance held by all five players at the end of the blockchain.

We decompose this task into three questions.

## Assignment Questions

**Question 1: Transaction validity.** Write the method `isValidTransaction` to verify the correctness of a `transaction` that is provided as input. Your function should return `True` if the transaction is valid and `False` otherwise.

A few reminders to help you out:

- Of the four properties listed above that determine whether a blockchain is valid, only the first property can be checked for an individual transaction. (The other three properties depend on the other contents in the block, or in other blocks of the blockchain, and we will address them in the next two questions.)

- In this toy blockchain, all signatures were created using ECDSA with a NIST-approved curve called `P-256`, and the message that is being signed is the hash of the transaction payload.

_Points:_ 2

In [41]:
def isValidTransaction(transaction: Transaction) -> bool:
    # Todo: write here!
    # return type: bool

    key = ECC.import_key(unhexlify(transaction.transaction_payload.sender_public_key), curve_name='P-256')
    h = transaction.transaction_payload.hash()
    verifier = DSS.new(key, 'fips-186-3')

    try:
        verifier.verify(h, unhexlify(transaction.signature))
        return True
    except ValueError:
        return False

In [42]:
isValidTransaction(blockchain[0].transactions[0])

True

In [43]:
grader.check("q1")

q1 results: All test cases passed!

**Question 2: Block validity.** Next, you must write the method `verifyBlock` to check that the provided `block` has been validly generated. Your function should return `True` if the block is valid and `False` otherwise.

That is: verify that each block satisfies the following two rules.

1. The signature in every transaction is correct, and
2. Every block has a `nonce` that is chosen to satisfy the difficulty rule.

(Note: we stated in the instructions above that blocks in a blockchain must satisfy _four_ rules. But these are the only two rules that can be checked by looking at a single block; the other two rules require you to look at the entire blockchain at once, and we will defer those checks to the final question of the assignment.)

_Points:_ 1

In [44]:
def verifyBlock(block: Block) -> bool:
    # Todo: write here!
    # return type: bool

    # (1) difficulty rule
    if block.hash()[:2] != "00":
        return False

    # (2) valid transaction rule
    for transaction in block.transactions:
        if not isValidTransaction(transaction):
            return False
        
    return True

In [45]:
verifyBlock(blockchain[0])

True

In [46]:
grader.check("q2")

q2 results: All test cases passed!

**Question 3: Blockchain validity and balance calculation.** Finally, write the method `findBalance` that determines if the entire blockchain is valid. 
- If so, then return the balance of the each player in the type of `list[int]`.
- If not, then return `False`.

This method should take two parameters as inputs:
- the `blockchain` that you are checking the validity of, and
- a list of integers containing the initial `players_balance` before the start of the blockchain. Since our toy blockchain doesn't have any mining rewards, instead we initialize the toy blockchain by asserting that Players A and C have one coin each before the start of the system.

Remember that you must check all four properties that determine whether a blockchain is valid:

1. The signature in every transaction is correct,
2. Every block has a `nonce` that is chosen to satisfy the difficulty rule,
3. Every block points to the previous block (and by convention, the initial block -- called the "genesis" block -- has a hash of all-0s), and
4. Everyone has enough money in their account at the moment they post a transaction.

You may use the functions `isValidTransaction` and `verifyBlock` that you created in questions 1 and 2.

_Points:_ 2

In [47]:
## Execute, but DO NOT MODIFY this code block. ##

from typing import Union
players_balance = [1,0,1,0,0] # initially, Players A and C have 1 coin each

In [48]:
def findBalance(blockchain: list[Block], players_balance: list[int]) -> Union[list[int], bool]:
    # input: a blockchain and an integer balance
    # output: either the new list of int, or the bool `False`

    # Todo: write here!
    # return type: list[int] or bool
    
    for block in blockchain:
        if not verifyBlock(block):
            return False

        for transaction in block.transactions:
            sender_index = public_keys.index(transaction.transaction_payload.sender_public_key)
            receiver_index = public_keys.index(transaction.transaction_payload.receiver_public_key)
            amount = transaction.transaction_payload.amount

            if players_balance[sender_index] < amount:
                return False

            players_balance[sender_index] -= amount
            players_balance[receiver_index] += amount
    
    return players_balance

In [49]:
grader.check("q3")

q3 results: All test cases passed!

## Submitting the Assignment

Please follow these instructions to complete the assignment and submit it for credit.

**Documenting collaborators, sources, and AI tools:** In accordance with the collaboration policy, use the space below to report if you used any resources to complete this homework assignment, aside from the lecture notes and the course textbooks/videos. Specifically, please report:

1. Names of all classmates you worked with, and a short description of the work that you performed together.
2. All written materials that you used, such as books or websites (besides the lecture notes or textbooks). Please include links to any web-based resources, or citations to any physical works.
3. All code that you used from other sources. In particular, if you used an AI tool, then you must include the entire exchange with the AI tool, as per the [CDS Generative AI Assistance Policy](https://www.bu.edu/cds-faculty/culture-community/gaia-policy/).

Remember that if we discover any undocumented collaborators, sources, or AI tools then this is grounds for a grade penalty and referral to BU's Academic Conduct Committee (as described in the syllabus).

_Your response:_

1. n/a

2. DSS and ECC documentation

3. n/a

**Sending to Gradescope:** After completing the assignment:
- if you did the assignment on Colab, download it in `.ipynb` format.
- if you did the assignment locally on your machine, all you need to do is to find it in your directory.

Then, submit only the `.ipynb` file to this week's programming assignment on Gradescope. It may take a few seconds or a minute for the auto-grading system to check your work.

Remember that you can submit as many times as you want until the deadline for the assignment; only your last score counts.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Submit your assignment on Gradescope once you have passed enough tests to receive at least 4 of the 5 available points.

In [51]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)

c:\Users\jackw\AppData\Local\Programs\Python\Python311\Lib\site-packages\otter\check\notebook.py:470: UserWarning: Could not locate a PDF to include
  warnings.warn("Could not locate a PDF to include")


UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 17531: character maps to <undefined>